This is the test code for BC agent in the CARLA simulator.

In [1]:
import carla 
import math 
import random 
import time 
import csv
import numpy as np
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

2024-12-10 14:36:19.550591: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 14:36:19.577939: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 14:36:19.577973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 14:36:19.578704: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 14:36:19.583320: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
saved_model_path = 'regression/IL_town06_steer_bs32_resnet.h5' 
loaded_model = tf.keras.models.load_model(saved_model_path)

2024-12-10 14:36:21.169174: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-10 14:36:21.169197: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: umd-002872
2024-12-10 14:36:21.169202: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: umd-002872
2024-12-10 14:36:21.169294: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-12-10 14:36:21.169311: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.183.1
2024-12-10 14:36:21.169314: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.183.1


In [4]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 
print(spawn_points)

[<carla.libcarla.Transform object at 0x7f22afb70cc0>, <carla.libcarla.Transform object at 0x7f22aed4bec0>, <carla.libcarla.Transform object at 0x7f22aed4bcc0>, <carla.libcarla.Transform object at 0x7f22aed4bf40>, <carla.libcarla.Transform object at 0x7f22afd0adc0>, <carla.libcarla.Transform object at 0x7f22ae234a40>, <carla.libcarla.Transform object at 0x7f22ae237840>, <carla.libcarla.Transform object at 0x7f22ae234bc0>, <carla.libcarla.Transform object at 0x7f22ae235040>, <carla.libcarla.Transform object at 0x7f22ae234140>, <carla.libcarla.Transform object at 0x7f22ae2345c0>, <carla.libcarla.Transform object at 0x7f22ae235bc0>, <carla.libcarla.Transform object at 0x7f22ae2376c0>, <carla.libcarla.Transform object at 0x7f22ae235440>, <carla.libcarla.Transform object at 0x7f22ae2377c0>, <carla.libcarla.Transform object at 0x7f22ae2344c0>, <carla.libcarla.Transform object at 0x7f22ae2355c0>, <carla.libcarla.Transform object at 0x7f22ae234fc0>, <carla.libcarla.Transform object at 0x7f22ae2

In [5]:
settings = world.get_settings()
settings.fixed_delta_seconds = 0.01
world.apply_settings(settings)

11720

In [6]:
# Add the ego vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[427])

# Move the spectator behind the vehicle to view it
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)


In [7]:
print(spawn_points[1])

Transform(Location(x=584.831238, y=-13.577756, z=0.300000), Rotation(pitch=0.000000, yaw=-179.580566, roll=0.000000))


change this part
#######################

In [8]:
# Set initial camera translation
""""change this part """
bound_x = 0.5 + vehicle.bounding_box.extent.x
bound_y = 0.5 + vehicle.bounding_box.extent.y
bound_z = 0.5 + vehicle.bounding_box.extent.z
#camera_init_trans = carla.Transform(carla.Location(x=+0.8*bound_x, y=+0.0*bound_y, z=1.3*bound_z))
camera_init_trans = carla.Transform(carla.Location(x=1.5, z=2.4), carla.Rotation(pitch=0))  # Front
# Add one of each type of camera
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)


In [9]:
# Define respective callbacks
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


In [10]:
def plot_image(model, image ):

    ## resize the input image 
    image = image[:,:,0:3]
    crop_coordinates = (350,719 , 0, 1279)

    # Crop the image using array slicing
    cropped_img = image[crop_coordinates[0]:crop_coordinates[1], crop_coordinates[2]:crop_coordinates[3]]
    image = cv2.resize(cropped_img, (160,160))
    image = np.array(image).astype("float32")/255.0
    image = np.reshape(image,(160,160,3))
    image = np.expand_dims(image, axis=0)
    X_train = image
    output= model.predict(X_train)

    return output

In [11]:
# Initialise parameters and data
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()



sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4))}

# OpenCV named window for display
cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

# Tile all data in one array
top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)

# Display with imshow
cv2.imshow('All cameras',top_row)
cv2.waitKey(1)

# Set sensors recording
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))

# Indefinite while loop
v2 = 0
acc = 0
steering = 0
brake = 0
cnt = 0
loc = []
with open('location2.csv', 'w', newline='') as file_command:
    writer = csv.writer(file_command,lineterminator='\n',)
    writer.writerow(['X',"Y","Z","pitch","yaw","roll","speed",'waypoint_x','waypoint_y','waypoint_z','width'])
    file_command.close()
while True:
    
    # Tile camera images into one array
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)
    position = (50, 50)  # (x, y) coordinates

    # Specify the font type
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Specify the font scale (font size)
    font_scale = 0.5

    # Specify the font color in BGR format
    font_color = (0, 0, 0)  # Blue

    # Specify the line type
    line_type = 2


    output_cmd = plot_image(loaded_model, sensor_data['rgb_image'])

    v = vehicle.get_velocity()
    tr = vehicle.get_transform()
    v2= 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
    carla_map = world.get_map()
    waypoint = carla_map.get_waypoint(tr.location, project_to_road=True, lane_type=carla.LaneType.Driving)
    width = waypoint.lane_width

    loc.append([tr.location.x,tr.location.y,tr.location.z,tr.rotation.pitch,tr.rotation.yaw,tr.rotation.roll,v2,waypoint.transform.location.x,waypoint.transform.location.y,waypoint.transform.location.z,width])
    
    

    steering =float(output_cmd[0])/4

    # Add the text to the image
    cv2.putText(top_row, 'velocity is: '+ f"{v2:.2f}" + '   throttle is: ' + str(acc) + '     steering is '+ f"{steering:.2f}"+ '     brake is '+ str(brake), position, font, font_scale, font_color, line_type)
    cv2.imshow('All cameras',top_row)


    if v2<30:
        acc=0.6
    else:
        acc=0
    
    vehicle.apply_control(carla.VehicleControl(throttle=acc, steer=steering))
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        # opening the csv file in 'w+' mode
        with open('location2.csv', 'a', newline='') as file:
            write = csv.writer(file)
            write.writerows(loc)
        break

# Stop sensors and destroy OpenCV window
camera.stop()
sem_camera.stop()


1/1 [==============================] - 0s 41ms/step


/tmp/ipykernel_520396/1540441713.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  steering =float(output_cmd[0])/4


1/1 [==============================] - 0s 46ms/step
